In [ ]:
%%javascript
    $("#homeButton").remove()
    $('body').append('<a href="#'+$("h1,h2,h3:eq(0)").attr("id")+'" style="position: fixed; bottom: 10px; right: 10px;" type="button" class="btn btn-success btn-circle btn-lg"><i class="glyphicon glyphicon-link" id="homeButton"></i></a>');
    $("#MainMenu").remove()
    var menu = 
      '<div id="MainMenu" style="position: fixed; top: 120px; right: 10px; z-index:6; max-height: 500px;">'+
        '<div class="list-group panel" >'+
          '<a href="#collapseMenu" class="list-group-item list-group-item-success" data-toggle="collapse" data-parent="#MainMenu">Indíce<i class="fa fa-caret-down"></i></a>'+
          '<div class="collapse" id="collapseMenu" style="overflow-y: overlay; max-height: 500px;">'+
          '</div>'+
        '</div>'+
      '</div>'
   
    var parent = $(menu)
    var arrayIds = []
    $("h1,h2,h3").attr("id",function(id,Value){
        if(Value != ""){
            var content = (Value.replace(new RegExp('-', 'g'), ' '));
            content = "&nbsp;".repeat(parseInt($(this).prop("tagName")[1])*2-1) + content;
            $(parent).find("#collapseMenu").append('<a href="#'+Value+'" style="position:relative;" class="list-group-item" data-parent="#SubMenu1">'+content+'</a>');
        }
    })
$('body').append(parent)

In [ ]:
%%html
<style>
    h3{
        color:#0094E4;
    }
    h2{
         color:#F28773;
    }
    h1{
         color:#0094E4;
    }
</style>

<img src="./images/logo.png" style="width: 40%; margin-left: 50%;">
<img src="./images/spark.png">

# Tipos de modelos y regresión lineal simple
En ciencia de datos o machine learning deseamos hacer programas que puedan aprender de los datos.

Una definición más formal y orientada a la ingeniería es:

<div class="alert alert-info">Se dice que un <strong>programa de computadora aprende de la experiencia E con respecto a una tarea T y alguna medida de desempeño P, si su desempeño en T, como medida de P, mejora con al experiencia E</strong>.</div>

Para aprender, nuestro programa usará un conjunto de datos llamados de "entrenamiento" o *training*. Este conjunto de datos es la experiencia **E** que le brindamos al programa. Un ejemplo de tarea **T** puede ser clasificar correos como spam o ham y la medida de desempeño podría ser el cociente de correos correctamente clasificados.

<div class="alert alert-danger" role="alert">
  Si bajamos una copia de Wikipedia a nuestra computadora tendremos más datos en ella, pero la computadora no se habrá vuelto mejor en ninguna tarea por lo que no se le puede considerar a esto Machine Learning ni Ciencia de Datos.
</div>

En general, usar técnicas de Machine Learning nos permite crear programas más pequeños, fáciles de mantener y que es más probable que nos den resultados correctos.

## Los tipos de modelos que se usan en ciencia de datos 


En general se clasifican, de acuerdo a la cantidad de supervisión que llevan durante el entrenamiento, en:
* **Supervisados**: Los datos de entrenamiento con los que alimentamos el algoritmo incluyen la solución deseada, llamada target (blanco) o label (etiqueta).


<img src="./images/training_sup.png" style="width: 60%; margin-left: 10%;">

* **No supervisados**: Los datos no están marcados (vienen sin etiqueta) y el sistema intenta aprender sin maestro.

<img src="./images/training_nosup.png" style="width: 85%; margin-left: 10%;">

## ¿Para qué usamos los modelos en ciencia de datos?


<img src="./images/class_vs_reg.png" style="width: 85%; margin-left: 10%;">

* **Clasificación:** Queremos mapear entradas a etiquetas salida
* **Regresión:** Queremos mapear una entrada a una salida continua. 


# Regresión lineal

Un modelo lineal hace una predicción computando una suma ponderada de las variables predictoras (input features), más una constante a la que se le llama el "término de sesgo" o término de intercepción.

$\hat{y}=\theta_{0}+\theta_{0}x_1+\theta_{2}x_2+...+\theta_{n}x_n$     

Donde:

* $\hat{y}:$ Es la predicción
* $n:$ Es el número de variables o características.
* $x_i:$ Es la iésima variable o característica.
* $\theta_j:$ Es el jotaésimo parámetro del modelo

Esta se puede expresar también de este modo:

$\hat{y}=\theta^{T} \cdot \mathbf{x}$     

Donde:

* $\theta:$ Es el vector de parámetros del modelo
* $\theta^{T}:$ Es el vector **traspuesto** de parámetros del modelo
* $\mathbf{x}:$ Es el vector de las variables   
* $\theta^{T} \cdot \mathbf{x}:$ Es el producto punto de $\theta^{T}$ y $\mathbf{x}$

Por lo general la P (medida de desempeño) que se usa aquí es el error cuadrado medio.

$ECM = \frac{1}{m}\sum_{i=0}^m(\theta^T \cdot x^{(i)}-y^{(i)})^2$


<img src="images/lineal_r_simple.png">

# Para verlo más claramente tomemos un problema:

https://www.kaggle.com/c/boston-housing/rules

Quiero conocer el valor medio de casa ocupadas por sus dueños en miles de dólares (**medv**) en los suburbios de Boston.

Mis variables (features, covariables,etc...) que represento como columnas son:

**crim**
tasa de crímenes *per capita* por ciudad.

**zn**
proporción de de tierras residenciales por lotes de 25,000 sq.ft.

**indus**
proportion of non-retail business acres per town.

**chas**
Charles River dummy variable (= 1 si estamos cerca del río; 0 e caos contrario).

**nox**
concentración de óxido de nitrógeno (partes por 10 millones).

**rm**
Número promedio de recámaras por casa.

**age**
proportion of owner-occupied units built prior to 1940.

**dis**
weighted mean of distances to five Boston employment centres.

**rad**
index of accessibility to radial highways.

**tax**
full-value property-tax rate per \$10,000.

**ptratio**
pupil-teacher ratio by town.

**black**
1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.

**lstat**
lower status of the population (percent).

**medv**
median value of owner-occupied homes in \$1000s.


## Cargamos los datos

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sc= SparkContext()
sqlContext = SQLContext(sc)

In [ ]:
house_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('./data/boston.csv')
house_df.take(1)

## Descriptivos de los datos (lo primero que debemos hacer siempre)

In [ ]:
house_df.printSchema()

In [ ]:
house_df.limit(10).toPandas()

In [ ]:
house_df.describe().toPandas().transpose()

## Supuestos de la regresión lineal

* **Linealidad:** Tus valores de $x$ y $y$ tienen una relación aproximadamente lineal.
* **Independencia:** Las covariables (features) $x_i$ y $x_j$ deben ser independientes para toda i ≠ j.

Dados los supuestos anteriores me gustaría ver si mis variables dependen unas de otras, por lo que hago una matriz de *scatter plots* (gráfica de dispersión).

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
numeric_features = [t[0] for t in house_df.dtypes if t[1] == 'int' or t[1] == 'double']
sampled_data = house_df.select(numeric_features).sample(False, 0.8).toPandas()
axs = pd.scatter_matrix(sampled_data, figsize=(10, 10))
n = len(sampled_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())


Otra cosa que podemos hacer es hallar la correlación entre las covariables y el target

In [ ]:
import six
for i in house_df.columns:
    if not( isinstance(house_df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlación con medv para ", i, house_df.stat.corr('medv',i))

<div class="alert alert-danger" role="alert">
  En el deber ser, tendríamos que filtrar todas las covariables que tienen una dependencia entre sí o combinarlas en una sola variable.
</div>

De momento dejaremos los datos como están.

## Preparando los datos para hacer nuestro modelo en Spark

In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['crim', 'zn',\
    'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',\
    'ptratio', 'black', 'lstat'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(house_df)
vhouse_df = vhouse_df.select(['features', 'medv'])
vhouse_df.show(3)

## Hago mis subconjuntos de entrenamiento y prueba

In [ ]:
splits = vhouse_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

### Ahora sí (por fin!) hago mi regresión lineal.

### 1.Entreno mi modelo con el subconjunto de entrenamiento

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='medv', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)

Veo mis coeficientes ($\theta$'s) y mi "intercept" $\theta_0$

In [ ]:
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Dados mis datos de entrenamiento, ¿cómo salió? Recordemos que para eso tengo mi criterio de error cuadrado medio.

In [ ]:
trainingSummary = lr_model.summary
print("ECM: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Prize time!!!!


<img src="./images/prize.jpeg">


¿Qué es $r^2$?

Todo se ve muy bien :D
    
Pero...

<div class="alert alert-danger" role="alert">
  El desempeño con mi conjunto de datos de entrenamiento puede no reflejar qué tan bien predice nuestro modelo lo que pasará con la población total o con otros conjunto de datos.
</div>

Entonces, usamos el conjutno de datos de prueba

### 2. Probamos con el conjunto de datos de prueba

In [ ]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","medv","features").show(5)

#### Evaluamos cómo quedó la predicción

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="medv",metricName="r2")
print("R cuadrada (r2) en datos de prueba = %g" % lr_evaluator.evaluate(lr_predictions))

In [ ]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="medv",metricName='rmse')

In [ ]:
print("ECM en datos de prueba = %g" % lr_evaluator.evaluate(lr_predictions))

### 3. Ahora, si tuvieramos otro conjunto de datos, el siguiente paso sería hacer predicciones con mi modelo

In [ ]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","medv","features").show()

## Fuentes
- https://www.kdnuggets.com/2018/04/supervised-vs-unsupervised-learning.html
- Hands-On Machine Learning with Scikit-Learn & Tensor Flow Auréliene Géron  
- https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a
- https://spark.apache.org/docs/2.2.0/ml-classification-regression.html#linear-regression
- https://medium.com/@mjspeck/a-guide-to-basic-modeling-techniques-properties-b2a88841e218